### Exploratory Download and Preprocessing of IMERG Monthly Precipitation Data

In this notebook, we download and preprocess the **IMERG Monthly Final Run (Version 07)** precipitation dataset.
The goal is to obtain a clean, analysis-ready monthly IMERG dataset for the period **2019–2025**, which will later
be compared against the GPCP monthly precipitation product.

This notebook focuses only on:
- downloading IMERG monthly files
- opening and concatenating them
- performing basic sanity checks
- saving a single processed NetCDF file

### Datasets Overview

#### IMERG (Integrated Multi-satellitE Retrievals for GPM)
- Monthly Final Run product
- Native resolution: ~0.1°
- Units: precipitation rate (mm/hr)
- Longitude convention: -180 to 180
- Strength: fine spatial detail

### GPCP (Global Precipitation Climatology Project)
- Monthly precipitation product
- Native resolution: 2.5°
- Units: precipitation rate (mm/day)
- Longitude convention: 0 to 360
- Strength: long-term climatological consistency

Given these differences, IMERG must be:
- temporally subset
- spatially subset
- unit-converted
- regridded to the GPCP grid


The monthly product is used to ensure consistency with the GPCP monthly dataset,
which is designed for climatological analysis.


In [1]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
import xarray as xr
import h5py
import requests
pd.set_option('display.max_rows', 100)


In [2]:
raw_dir = Path("../data/raw/imerg_monthly")
raw_dir.mkdir(parents=True, exist_ok=True)

raw_dir

WindowsPath('../data/raw/imerg_monthly')

In [3]:
from bs4 import BeautifulSoup

def list_imerg_monthly_files(year):
    url = f"https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGM.07/{year}/"
    
    r = requests.get(url)
    r.raise_for_status()
    
    soup = BeautifulSoup(r.text, "html.parser")
    
    files = [
        link.get("href")
        for link in soup.find_all("a")
        if link.get("href", "").endswith(".HDF5")
    ]
    
    return files


# Test for one year
list_imerg_monthly_files(2019)[:5]

['3B-MO.MS.MRG.3IMERG.20190101-S000000-E235959.01.V07B.HDF5',
 '3B-MO.MS.MRG.3IMERG.20190101-S000000-E235959.01.V07B.HDF5',
 '3B-MO.MS.MRG.3IMERG.20190201-S000000-E235959.02.V07B.HDF5',
 '3B-MO.MS.MRG.3IMERG.20190201-S000000-E235959.02.V07B.HDF5',
 '3B-MO.MS.MRG.3IMERG.20190301-S000000-E235959.03.V07B.HDF5']

In [4]:
base_url = "https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGM.07"
session = requests.Session()
session.trust_env = True  # uses .netrc

downloaded_files = []

for year in range(2019, 2022):
    try:
        files = list_imerg_monthly_files(year)
    except Exception as e:
        print(f"Could not list files for {year}: {e}")
        continue

    for fname in files:
        url = f"{base_url}/{year}/{fname}"
        out_file = raw_dir / fname

        if out_file.exists():
            downloaded_files.append(out_file)
            continue

        try:
            r = session.get(url, stream=True, timeout=60)
            r.raise_for_status()

            with open(out_file, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)

            downloaded_files.append(out_file)

        except Exception as e:
            print(f"Skipping {fname}: {e}")


In [5]:
# Local data directory (already downloaded IMERG monthly files)
raw_dir = Path('../data/raw/imerg_monthly')
files = sorted(raw_dir.glob('*.HDF5'))

print(f'Found {len(files)} files in {raw_dir.resolve()}')
print('First file:', files[0].name if files else 'None')
print('Last file :', files[-1].name if files else 'None')


Found 36 files in D:\esdp\ESDP-final-project\data\raw\imerg_monthly
First file: 3B-MO.MS.MRG.3IMERG.20190101-S000000-E235959.01.V07B.HDF5
Last file : 3B-MO.MS.MRG.3IMERG.20211201-S000000-E235959.12.V07B.HDF5


In [6]:
# Parse month from filename and check for gaps
def parse_month_from_name(name: str) -> pd.Timestamp:
    m = re.search(r'\.(\d{8})-S', name)
    if not m:
        raise ValueError(f'Cannot parse date from {name}')
    return pd.to_datetime(m.group(1), format='%Y%m%d')

file_months = pd.DatetimeIndex([parse_month_from_name(p.name) for p in files])
expected = pd.date_range(file_months.min(), file_months.max(), freq='MS')
missing = expected.difference(file_months)

print('Date span:', file_months.min().date(), 'to', file_months.max().date())
print('Expected months:', len(expected))
print('Available months:', len(file_months))
print('Missing months:', len(missing))
print(missing.tolist() if len(missing) else 'None')


Date span: 2019-01-01 to 2021-12-01
Expected months: 36
Available months: 36
Missing months: 0
None


In [7]:
# Inspect one sample HDF5 file structure
sample = files[0]
print('Sample file:', sample.name)

with h5py.File(sample, 'r') as f:
    datasets = []
    def walk(name, obj):
        if isinstance(obj, h5py.Dataset):
            datasets.append((name, obj.shape, str(obj.dtype)))
    f.visititems(walk)

for name, shape, dtype in datasets:
    print(f'{name:45s} shape={str(shape):18s} dtype={dtype}')


Sample file: 3B-MO.MS.MRG.3IMERG.20190101-S000000-E235959.01.V07B.HDF5
Grid/gaugeRelativeWeighting                   shape=(1, 3600, 1800)    dtype=int16
Grid/lat                                      shape=(1800,)            dtype=float32
Grid/lat_bnds                                 shape=(1800, 2)          dtype=float32
Grid/latv                                     shape=(2,)               dtype=int32
Grid/lon                                      shape=(3600,)            dtype=float32
Grid/lon_bnds                                 shape=(3600, 2)          dtype=float32
Grid/lonv                                     shape=(2,)               dtype=int32
Grid/nv                                       shape=(2,)               dtype=int32
Grid/precipitation                            shape=(1, 3600, 1800)    dtype=float32
Grid/precipitationQualityIndex                shape=(1, 3600, 1800)    dtype=float32
Grid/probabilityLiquidPrecipitation           shape=(1, 3600, 1800)    dtype=int16
Grid

#### Inspecting a Single IMERG Monthly File

To understand the structure of the IMERG monthly dataset, a single file
is opened and inspected for variables, coordinates, units, and grid layout.


In [8]:
# Open one sample with xarray (IMERG science variables live in group='Grid')
ds1 = xr.open_dataset(sample, engine='netcdf4', group='Grid')

print(ds1)
print('Variables:', list(ds1.data_vars))
print('Coordinates:', list(ds1.coords))
print('Precip units:', ds1['precipitation'].attrs.get('units'))

ds1.close()


<xarray.Dataset> Size: 130MB
Dimensions:                         (time: 1, lon: 3600, lat: 1800, nv: 2,
                                     lonv: 2, latv: 2)
Coordinates:
  * time                            (time) object 8B 2019-01-01 00:00:00
  * lon                             (lon) float32 14kB -179.9 -179.9 ... 179.9
  * lat                             (lat) float32 7kB -89.95 -89.85 ... 89.95
Dimensions without coordinates: nv, lonv, latv
Data variables:
    time_bnds                       (time, nv) object 16B ...
    lon_bnds                        (lon, lonv) float32 29kB ...
    lat_bnds                        (lat, latv) float32 14kB ...
    precipitation                   (time, lon, lat) float32 26MB ...
    randomError                     (time, lon, lat) float32 26MB ...
    gaugeRelativeWeighting          (time, lon, lat) float32 26MB ...
    probabilityLiquidPrecipitation  (time, lon, lat) float32 26MB ...
    precipitationQualityIndex       (time, lon, lat) float32 26

The IMERG monthly files are concatenated here only to verify temporal
continuity across the selected period.

This step is exploratory and no processed output is saved.
The full concatenation and subsetting will be implemented in scripts.


In [9]:
imerg = xr.open_mfdataset(
    files,
    engine="netcdf4",
    group="Grid",
    combine="by_coords",
    data_vars="minimal",
    coords="minimal",
    compat="override",
)

pr_imerg = imerg["precipitation"].transpose("time", "lat", "lon")
pr_imerg

<xarray.DataArray 'precipitation' (time: 36, lat: 1800, lon: 3600)> Size: 933MB
dask.array<transpose, shape=(36, 1800, 3600), dtype=float32, chunksize=(1, 1800, 145), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 288B 2019-01-01 00:00:00 ... 2021-12-01 00:00:00
  * lon      (lon) float32 14kB -179.9 -179.9 -179.8 ... 179.8 179.8 179.9
  * lat      (lat) float32 7kB -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
Attributes:
    DimensionNames:    time,lon,lat
    Units:             mm/hr
    units:             mm/hr
    CodeMissingValue:  -9999.9
    LongName:          Merged microwave-infrared-gauge precipitation estimate

### GPCP Monthly Precipitation Dataset

The GPCP monthly mean precipitation dataset was downloaded manually
from the NOAA PSL archive and placed in the `data/processed/` directory.

This dataset is distributed as a single NetCDF file and does not
require authenticated access or an API for download.


In [10]:
ds_gpcp = xr.open_dataset("../data/processed/precip.mon.mean.nc")
ds_gpcp

<xarray.Dataset> Size: 23MB
Dimensions:    (lat: 72, lon: 144, time: 564, nv: 2)
Coordinates:
  * lat        (lat) float32 288B -88.75 -86.25 -83.75 ... 83.75 86.25 88.75
  * lon        (lon) float32 576B 1.25 3.75 6.25 8.75 ... 353.8 356.2 358.8
  * time       (time) datetime64[ns] 5kB 1979-01-01 1979-02-01 ... 2025-12-01
Dimensions without coordinates: nv
Data variables:
    time_bnds  (time, nv) datetime64[ns] 9kB ...
    lat_bnds   (lat, nv) float32 576B ...
    lon_bnds   (lon, nv) float32 1kB ...
    precip     (time, lat, lon) float32 23MB ...
Attributes: (12/18)
    Conventions:           CF-1.0
    curator:               Dr. Jian-Jian Wang\nESSIC, University of Maryland ...
    citation:              Adler, R.F., G.J. Huffman, A. Chang, R. Ferraro, P...
    title:                 GPCP Version 2.3 Combined Precipitation Dataset (F...
    platform:              NOAA POES (Polar Orbiting Environmental Satellites)
    source_obs:            CDR RSS SSMI/SSMIS Tbs over ocean \nCDR SSMI/SSMIS...
    ...                    ...
    source:                https://www.ncei.noaa.gov/data/global-precipitatio...
    source_documentation:  https://www.ncdc.noaa.gov/cdr/atmospheric/precipit...
    NCO:                   4.6.9
    history:               Generated at NOAA/ESRL PSD Sep 9 2016 based on dat...
    References:            http://www.psl.noaa.gov/data/gridded/data.gpcp.html
    data_comment:          Interim data covers 2025/11 through latest.

From inspection of the GPCP dataset, the following characteristics
are observed:

- Temporal resolution: monthly
- Spatial resolution: 2.5° × 2.5°
- Longitude convention: 1.25° to 358.75° (0–360 grid, cell-centered)
- Precipitation units: mm/day
- Temporal coverage extends beyond the IMERG period
